In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# General imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import regularizers
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import load_model
import seaborn as sns
from mlxtend.plotting import plot_decision_regions
from sklearn.metrics import confusion_matrix
import cv2
import PIL

In [ ]:
# Set up ImageDataGenerator
train_imagegen = keras.preprocessing.image.ImageDataGenerator(rescale=1./255,
                                   zoom_range=[0.6,1],
                                   rotation_range=10,
                                   brightness_range=([0.6, 1.5]),
                                   horizontal_flip=True,
                                   validation_split=0.06) # this will set aside a part of training set for validation data
test_imagegen = keras.preprocessing.image.ImageDataGenerator(rescale=1./255,
                                   zoom_range=[0.6,1],
                                   rotation_range=10,
                                   brightness_range=([0.6, 1.5]),
                                   horizontal_flip=True)
# Bring the data in
train_generator = train_imagegen.flow_from_directory(
                                    '../input/brain-tumor-classification-mri/Training',
                                    target_size=(200,200),
                                    batch_size=20,
                                    class_mode='categorical',
                                    subset='training')

test_generator = test_imagegen.flow_from_directory(
                                    '../input/brain-tumor-classification-mri/Testing',
                                    target_size=(200,200),
                                    batch_size=20,
                                    class_mode='categorical')

val_generator = train_imagegen.flow_from_directory(
                                    '../input/brain-tumor-classification-mri/Training',
                                    target_size=(200,200),
                                    batch_size=20,
                                    class_mode='categorical',
                                    subset='validation')

In [ ]:
def visualize_training_results(history):
    '''
    From https://machinelearningmastery.com/display-deep-learning-model-training-history-in-keras/
    
    Input: keras history object (output from trained model)
    '''
    fig, (ax1, ax2) = plt.subplots(2, sharex=True)
    fig.suptitle('Model Results')

    # summarize history for accuracy
    ax1.plot(history.history['acc'])
    ax1.plot(history.history['val_acc'])
    ax1.set_ylabel('Accuracy')
    ax1.legend(['train', 'test'], loc='upper left')
    # summarize history for loss
    ax2.plot(history.history['loss'])
    ax2.plot(history.history['val_loss'])
    ax2.set_ylabel('Loss')
    ax2.legend(['train', 'test'], loc='upper left')
    
    plt.xlabel('Epoch')
    plt.show()

In [ ]:
# Visualize (code from https://github.com/austint1121/OES-PneumoniaClassification/blob/main/Final_Notebook.ipynb)
train_batch = train_generator.next()
fig, axes = plt.subplots(2, 5, figsize=(16, 8))
    
for i in range(10):
    # Load image into numpy array and re-scale
    img = np.array(train_batch[0][i] * 255, dtype='uint8')
    ax = axes[i // 5, i % 5]
    ax.imshow(img)
fig.suptitle('Training Images')
plt.tight_layout()
plt.show()

## **Getting a look at the Distribution of Different Tumor Types**

In [ ]:
train_generator.mode

In [ ]:
train_generator.class_indices

In [ ]:
train_generator.classes

In [ ]:
train_tumor_types = pd.DataFrame(train_generator.classes)
train_values = train_tumor_types.value_counts()
train_values

In [ ]:
train_tumor_types.rename(columns={0:'Tumor Type'}, inplace=True)

In [ ]:
# Preparations for visualization
no_tumor = len(train_tumor_types[train_tumor_types['Tumor Type'] == 2])
glioma = len(train_tumor_types[train_tumor_types['Tumor Type'] == 0])
meningioma = len(train_tumor_types[train_tumor_types['Tumor Type'] == 1])
pituitary = len(train_tumor_types[train_tumor_types['Tumor Type'] == 3])

In [ ]:

fig, ax = plt.subplots(figsize=(10,8))
ax.bar(x=['No Tumor', 'Glioma', 'Meningioma', 'Pituitary'], height = [no_tumor, glioma, meningioma, pituitary])
ax.set(xlabel='', ylabel='Number of Images', title='Distribution of Brain Tumor Type');


## **Taking a Look at Different Tumor Types**

In [ ]:
# View a Glioma tumor
glioma1 = PIL.Image.open('../input/brain-tumor-classification-mri/Training/glioma_tumor/gg (1).jpg')
glioma1

In [ ]:
glioma1.mode

In [ ]:
glioma1.size

## **First Baseline Model**

In [ ]:
baseline = keras.Sequential()
baseline.add(layers.Conv2D(32, (3,3), activation='relu', input_shape=(200,200,3)))
baseline.add(layers.MaxPooling2D(2,2))
baseline.add(layers.Conv2D(64, (3,3), activation='relu'))
baseline.add(layers.MaxPooling2D(2,2))

baseline.add(layers.Flatten())
baseline.add(layers.Dense(128, activation='relu'))
baseline.add(layers.Dense(4, activation='softmax'))

baseline.compile(loss='categorical_crossentropy',
                optimizer='adam',
                metrics=['acc', 'Recall', 'Precision', 'TruePositives', 'TrueNegatives', 'FalsePositives', 'FalseNegatives'])

baseline_results = baseline.fit_generator(train_generator,
                                         steps_per_epoch=2699/20,# number of samples / batch size
                                         epochs=10,
                                         validation_data=test_generator)

In [ ]:
visualize_training_results(baseline_results)

## **Adding Class Weights to Baseline CNN**

In [ ]:
base_weights = keras.Sequential()
base_weights.add(layers.Conv2D(32, (3,3), activation='relu', input_shape=(200,200,3)))
base_weights.add(layers.MaxPooling2D(2,2))
base_weights.add(layers.Conv2D(64, (3,3), activation='relu'))
base_weights.add(layers.MaxPooling2D(2,2))

base_weights.add(layers.Flatten())
base_weights.add(layers.Dense(128, activation='relu'))
base_weights.add(layers.Dense(4, activation='softmax'))

base_weights.compile(loss='categorical_crossentropy',
                optimizer='adam',
                metrics=['acc', 'Recall', 'Precision', 'TruePositives', 'TrueNegatives', 'FalsePositives', 'FalseNegatives'])

multi_weights = {0: 1,
                1: 1 ,
                2: 2,
                3: 1,}

base_weights_results = base_weights.fit_generator(train_generator,
                                          class_weight=multi_weights,
                                         steps_per_epoch=2699/20,# number of samples / batch size
                                         epochs=10,
                                         validation_data=test_generator)


## **Adding Batch Normalization**

In [ ]:
norm = keras.Sequential()
norm.add(layers.Conv2D(32, (3,3), activation='relu', input_shape=(200,200,3)))
norm.add(layers.BatchNormalization())
norm.add(layers.MaxPooling2D(2,2))
norm.add(layers.Conv2D(64, (3,3), activation='relu'))
norm.add(layers.BatchNormalization())
norm.add(layers.MaxPooling2D(2,2))

norm.add(layers.Flatten())
norm.add(layers.Dense(128, activation='relu'))
norm.add(layers.BatchNormalization())
norm.add(layers.Dense(4, activation='softmax'))

norm.compile(loss='categorical_crossentropy',
                optimizer='adam',
                metrics=['acc', 'Recall', 'Precision', 'TruePositives', 'TrueNegatives', 'FalsePositives', 'FalseNegatives'])

multi_weights = {0: 1,
                1: 1 ,
                2: 2,
                3: 1,}

norm_results = norm.fit_generator(train_generator,
                                          class_weight=multi_weights,
                                         steps_per_epoch=2699/20,# number of samples / batch size
                                         epochs=10,
                                         validation_data=test_generator)

In [ ]:
visualize_training_results(norm_results)